# 🗂️ Explorando o Data Lake (Parquet)

Neste notebook vamos explorar os dados armazenados no **Data Lake** (arquivos Parquet puros).

O objetivo é demonstrar como funciona um Data Lake tradicional:
- Sem transações ACID
- Sem versionamento
- Sem schema enforcement
- Leitura direta dos arquivos `.parquet`

In [ ]:
import duckdb
import os

# Paths
BASE_DIR = os.path.abspath('..')
BRONZE = os.path.join(BASE_DIR, 'data', 'lake', '01_bronze')
SILVER = os.path.join(BASE_DIR, 'data', 'lake', '02_silver')
GOLD = os.path.join(BASE_DIR, 'data', 'lake', '03_gold')

con = duckdb.connect()

## 1. Bronze Layer
Dados brutos convertidos de CSV para Parquet, sem nenhuma transformação.

In [ ]:
# Verificar os arquivos na pasta Bronze
print('Arquivos em Bronze:')
for f in os.listdir(BRONZE):
    size_mb = os.path.getsize(os.path.join(BRONZE, f)) / (1024 * 1024)
    print(f'  {f} ({size_mb:.2f} MB)')

In [ ]:
# Ler e explorar o Parquet da Bronze
bronze_path = os.path.join(BRONZE, 'online_retail.parquet')

con.execute(f"""
    SELECT * FROM read_parquet('{bronze_path}') LIMIT 5
""").df()

In [ ]:
# Schema dos dados
con.execute(f"""
    DESCRIBE SELECT * FROM read_parquet('{bronze_path}')
""").df()

In [ ]:
# Contagem de registros
con.execute(f"""
    SELECT COUNT(*) as total_rows FROM read_parquet('{bronze_path}')
""").df()

## 2. Silver Layer
Dados limpos e modelados em Star Schema (Fato + Dimensões).

In [ ]:
# Verificar arquivos na Silver
print('Arquivos em Silver:')
for f in os.listdir(SILVER):
    size_mb = os.path.getsize(os.path.join(SILVER, f)) / (1024 * 1024)
    print(f'  {f} ({size_mb:.2f} MB)')

In [ ]:
# Fact Sales
fact_path = os.path.join(SILVER, 'fact_sales.parquet')
con.execute(f"SELECT * FROM read_parquet('{fact_path}') LIMIT 5").df()

In [ ]:
# Dim Customer
cust_path = os.path.join(SILVER, 'dim_customer.parquet')
con.execute(f"SELECT * FROM read_parquet('{cust_path}') LIMIT 5").df()

In [ ]:
# Dim Product
prod_path = os.path.join(SILVER, 'dim_product.parquet')
con.execute(f"SELECT * FROM read_parquet('{prod_path}') LIMIT 5").df()

In [ ]:
# Contagem de registros por tabela
for table in ['fact_sales', 'dim_customer', 'dim_product']:
    path = os.path.join(SILVER, f'{table}.parquet')
    count = con.execute(f"SELECT COUNT(*) FROM read_parquet('{path}')").fetchone()[0]
    print(f'{table}: {count:,} registros')

## 3. Gold Layer
Dados agregados prontos para consumo analítico.

In [ ]:
# Daily Sales
daily_path = os.path.join(GOLD, 'daily_sales.parquet')
con.execute(f"SELECT * FROM read_parquet('{daily_path}') LIMIT 10").df()

In [ ]:
# Sales by Country
country_path = os.path.join(GOLD, 'sales_by_country.parquet')
con.execute(f"SELECT * FROM read_parquet('{country_path}') LIMIT 10").df()

## ⚠️ Limitações do Data Lake (Parquet Puro)

Vamos demonstrar algumas limitações:

In [ ]:
# 1. Sem versionamento / time travel
# Se sobrescrevermos o arquivo, perdemos os dados antigos
print('No Data Lake não existe conceito de versão.')
print('Se sobrescrevermos o parquet, os dados antigos são perdidos.')
print('Não há como fazer "SELECT * FROM tabela VERSION AS OF 1"')

In [ ]:
# 2. Sem schema enforcement — qualquer dado pode ser escrito
import pandas as pd

df_wrong_schema = pd.DataFrame({
    'coluna_errada': [1, 2, 3],
    'outra_coluna': ['a', 'b', 'c']
})

# Isto vai funcionar! O Parquet aceita qualquer schema.
wrong_path = os.path.join(BRONZE, 'dados_errados.parquet')
df_wrong_schema.to_parquet(wrong_path, index=False)
print(f'Arquivo com schema errado escrito em: {wrong_path}')
print('O Data Lake não impede a escrita de dados com schema diferente!')

# Limpeza
os.remove(wrong_path)
print('(Arquivo removido para manter a pasta limpa)')

In [ ]:
# 3. Sem transações ACID
print('Se o processo falhar no meio da escrita, o arquivo fica corrompido.')
print('Não há rollback automático no Data Lake.')